In [64]:
import sys
sys.path += ["../src"]
import utils
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt 
import seaborn as sns
from matplotlib.pyplot import subplots as sbp 
from importlib import reload
import jl_vae
# import jl_nflows_geo_coordinates_2 as nfg
# from jl_nflows_geo_coordinates import load_nf as load_dict

from _51_abm_functions import cod_prov_abbrv_df

In [65]:
# transform one-hot encoding to categories
def add_cat_features(df):
    df["energy_class"] = df[[u for u in df.columns if "_energy" in u]].stack().rename("col").reset_index().query("col == 1")["level_1"]
    df["COD_CAT"] = [u[8:] for u in df[[u for u in df.columns if "COD_CAT_" in u]].stack().rename("col").reset_index().query("col == 1")["level_1"]]
    df["anno_costruzione"] = [u[17:] for u in df[[u for u in df.columns if "ANNO_COSTRUZIONE" in u]].stack().rename("col").reset_index().query("col == 1")["level_1"]]
    return df

In [66]:
# Load data from Jacopo's syn pops

# import dictionary with data {'hydro_risk', 'census', 'omi_og', 'cap'}
# takes ~25seconds
geo_dict = jl_vae.load_geo_data()

# check which provinces are done
glob(jl_vae.path_pop_synth + f"95sample/pop_samples/*")

['/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples/synthetic_pop_full_250806priceFE.csv',
 '/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples/synthetic_pop_full_250703LT.csv',
 '/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples/synthetic_pop_full_250703FI.csv',
 '/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples/synthetic_pop_full_250703CE.csv',
 '/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples/synthetic_pop_full_250703BT.csv',
 '/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples/synthetic_pop_full_250806priceKR.csv',
 '/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples/synthetic_pop_full_250703SI.csv',
 '/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples/synthetic_pop_full_250703CO.csv',
 '/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples/synthetic_pop_full_250811priceMC.csv',
 '/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples

In [67]:
import pickle

prov = "AN"

with open(f'/data/housing/data/intermediate/jl_pop_synth/isp_baselines/all_baselines_{prov}.pickle', 'rb') as f:
    all_baselines = pickle.load(f)

dummy_cols_list = ['flag_garage', 'flag_pertinenza', 'flag_air_conditioning',
    'flag_multi_floor', 'ANNO_COSTRUZIONE_1500_1965',
    'ANNO_COSTRUZIONE_1965_1985', 'ANNO_COSTRUZIONE_1985_2005',
    'ANNO_COSTRUZIONE_2005_2025', 'ANNO_COSTRUZIONE_Missing',
    'High_energy_class', 'Low_energy_class', 'Medium_energy_class',
    'Missing_energy_class', 'COD_CAT_A02', 'COD_CAT_A03',
    'COD_CAT_A_01_07_08', 'COD_CAT_A_04_05', 'floor_0.0', 'floor_1.0',
    'floor_2.0', 'floor_3.0', 'floor_Missing', 'floor_plus_4',
    'flag_air_conditioning_Missing', 'flag_multi_floor_Missing']

coord_cols_list = ['x','y']
cont_cols_list = ['log_mq','log_price']

In [97]:
# function returning the dataframes containing populations with continuous columns binned
def ConvertBool2number(df):
    bool_cols = df.select_dtypes(include=bool).columns
    df[bool_cols] = df[bool_cols].astype(float)
    df = df.reset_index(drop=True)
    return df

def BinnedContColumns(df_syn,df_real,vars):
    df = df_syn.copy()
    for var in vars:
        bins_values = pd.qcut(df_real[var], q = 5, labels = False, duplicates = "drop", retbins = True)[1]
        # in the following 2 rows I make that mq and prices lower than the real ones go inside the same bins
        bins_values[0] = -np.inf
        bins_values[-1] = np.inf
        df[var+'_bin'] = pd.cut(x = df_syn[var],bins = bins_values, labels=[0,1,2,3,4],include_lowest=True)
        
        df.drop(columns = [var], inplace = True)

    return df

def Transform_bins(df, vars):
    df1 = df.copy()
    for var in vars:
        df1[var + "_bin"] =  pd.qcut(df1[var], q = 5, labels = [0,1,2,3,4], duplicates = "drop").astype(int)
        df1.drop(columns = [var], inplace = True)
    return df1

def DataPreparation(data,keys,cont_cols,geo_dict):
    data_prep = dict() # output dictionary
    for key in keys: # loop across keys
        df = data[key].copy() # selecting the data

        # convert bool to integers
        df = ConvertBool2number(df)

        # binning the continuous columns
        if 'real' in key:
            df = Transform_bins(df, cont_cols)
        else:
            df = BinnedContColumns(df,data['df_real'],cont_cols)

        # add CAP and other stuff
        df = utils.spatial_matching_ABM(df, geo_dict["hydro_risk"], geo_dict["census"], 
                                        geo_dict["omi_og"], geo_dict["cap"])
        
        data_prep[key] = df
    
    return data_prep
            

In [98]:
keys_list = ['df_real', 'df_nfvae', 'df_ablation', 'df_ipf', 'df_copula_nf', 'df_copula_ablation']
keys_list95 = ['df_real95', 'df_nfvae95', 'df_ablation95', 'df_ipf95', 'df_copula_nf95', 'df_copula_ablation95']

In [99]:
data = DataPreparation(data = all_baselines,keys=keys_list,
                    cont_cols=cont_cols_list,geo_dict=geo_dict)

In [100]:
data['df_real'].columns

Index(['flag_garage', 'flag_pertinenza', 'flag_air_conditioning',
       'flag_multi_floor', 'GEO_LATITUDINE_BENE_ROUNDED',
       'GEO_LONGITUDINE_BENE_ROUNDED', 'ANNO_COSTRUZIONE_1500_1965',
       'ANNO_COSTRUZIONE_1965_1985', 'ANNO_COSTRUZIONE_1985_2005',
       'ANNO_COSTRUZIONE_2005_2025', 'ANNO_COSTRUZIONE_Missing',
       'High_energy_class', 'Low_energy_class', 'Medium_energy_class',
       'Missing_energy_class', 'COD_CAT_A02', 'COD_CAT_A03',
       'COD_CAT_A_01_07_08', 'COD_CAT_A_04_05', 'floor_0.0', 'floor_1.0',
       'floor_2.0', 'floor_3.0', 'floor_Missing', 'floor_plus_4',
       'flag_air_conditioning_Missing', 'flag_multi_floor_Missing',
       'log_mq_bin', 'log_price_bin', 'COD_CONTRATTO', 'SEZ2011', 'PRO_COM',
       'flag_geo_valid', 'CAP', 'OMI_id', 'OMI_categ', 'scenario',
       'scenario_LowRisk', 'scenario_MediumRisk', 'scenario_NoRisk',
       'scenario_HighRisk', 'scenario_Risk', 'flag_italy', 'COD_PROV',
       'COD_REG', 'regione_nome', 'prov_nome', 'pro

In [101]:
for key in data.keys():
    print(key,' Num homes with flag_geo_flaid = 1: ', data[key].flag_geo_valid.sum())

df_real  Num homes with flag_geo_flaid = 1:  4252
df_nfvae  Num homes with flag_geo_flaid = 1:  4252
df_ablation  Num homes with flag_geo_flaid = 1:  4252
df_ipf  Num homes with flag_geo_flaid = 1:  4252
df_copula_nf  Num homes with flag_geo_flaid = 1:  4252
df_copula_ablation  Num homes with flag_geo_flaid = 1:  4252


In [102]:
# drop useless columns
col2drop = ['COD_CONTRATTO', 'SEZ2011', 'PRO_COM',
       'flag_geo_valid', 'OMI_id', 'OMI_categ', 'scenario',
       'scenario_LowRisk', 'scenario_MediumRisk', 'scenario_NoRisk',
       'scenario_HighRisk', 'scenario_Risk', 'flag_italy', 'COD_PROV',
       'COD_REG', 'regione_nome', 'prov_nome', 'prov_abbrv',
       'year_erogaz_prov']

for key in data.keys():
    data[key] = data[key].drop(columns=col2drop)

In [103]:
data['df_real'].columns

Index(['flag_garage', 'flag_pertinenza', 'flag_air_conditioning',
       'flag_multi_floor', 'GEO_LATITUDINE_BENE_ROUNDED',
       'GEO_LONGITUDINE_BENE_ROUNDED', 'ANNO_COSTRUZIONE_1500_1965',
       'ANNO_COSTRUZIONE_1965_1985', 'ANNO_COSTRUZIONE_1985_2005',
       'ANNO_COSTRUZIONE_2005_2025', 'ANNO_COSTRUZIONE_Missing',
       'High_energy_class', 'Low_energy_class', 'Medium_energy_class',
       'Missing_energy_class', 'COD_CAT_A02', 'COD_CAT_A03',
       'COD_CAT_A_01_07_08', 'COD_CAT_A_04_05', 'floor_0.0', 'floor_1.0',
       'floor_2.0', 'floor_3.0', 'floor_Missing', 'floor_plus_4',
       'flag_air_conditioning_Missing', 'flag_multi_floor_Missing',
       'log_mq_bin', 'log_price_bin', 'CAP'],
      dtype='object')

In [104]:
# list of all CAP codes in the selected province
cap_list = []
for key in data.keys():
    cap_list.append(data[key].CAP.unique())

cap = np.unique(np.concatenate(cap_list))
cap

array(['60010', '60011', '60012', '60013', '60015', '60018', '60019',
       '60020', '60021', '60022', '60024', '60025', '60026', '60027',
       '60030', '60031', '60033', '60034', '60035', '60036', '60037',
       '60038', '60039', '60040', '60041', '60043', '60044', '60048',
       '60121', '60122', '60123', '60124', '60125', '60126', '60127',
       '60128', '60129', '60131'], dtype=object)

In [105]:
# passare da dummy variables to categorical columns
import pandas as pd

def dummies_to_category(df, cols, new_col, mapping=None, drop=True):
    """
    Converte un insieme di variabili dummy in una colonna categorica.

    Parametri
    ----------
    df : pd.DataFrame
        Il dataframe di input.
    cols : list
        Lista delle colonne dummy da combinare.
    new_col : str
        Nome della nuova colonna categorica.
    mapping : dict, opzionale
        Dizionario per rinominare le categorie {dummy_col: categoria}.
    drop : bool, default True
        Se True elimina le colonne dummy originali.

    Ritorna
    -------
    pd.DataFrame
        DataFrame con la nuova colonna categorica.
    """
    df = df.copy()
    
    # trova la colonna con il valore 1
    df[new_col] = df[cols].idxmax(axis=1)
    
    # se c'è un mapping, rinomina le categorie
    if mapping is not None:
        df[new_col] = df[new_col].map(mapping)
    
    # eventualmente elimina le colonne dummy
    if drop:
        df.drop(columns=cols, inplace=True)
    
    return df


In [106]:
energy_cols = ['High_energy_class', 'Low_energy_class', 'Medium_energy_class','Missing_energy_class']
floor_cols = ['floor_0.0', 'floor_1.0', 'floor_2.0', 'floor_3.0', 'floor_Missing', 'floor_plus_4']
cod_cat_cols = ['COD_CAT_A02', 'COD_CAT_A03', 'COD_CAT_A_01_07_08', 'COD_CAT_A_04_05']
anno_costr_cols = ['ANNO_COSTRUZIONE_1500_1965','ANNO_COSTRUZIONE_1965_1985', 'ANNO_COSTRUZIONE_1985_2005',
       'ANNO_COSTRUZIONE_2005_2025', 'ANNO_COSTRUZIONE_Missing']
ac_cols = ['flag_air_conditioning','flag_air_conditioning_Missing']
multi_floor_cols = ['flag_multi_floor','flag_multi_floor_Missing']

In [107]:
data_rev = dict()
for key in data.keys():
    df=data[key].copy()
    df = dummies_to_category(df,energy_cols,'energy_class')
    df = dummies_to_category(df,floor_cols,'floor')
    df = dummies_to_category(df,cod_cat_cols,'cod_cat')
    df = dummies_to_category(df,anno_costr_cols,'costr_year')
    df = dummies_to_category(df,ac_cols,'air_conditioning')
    df = dummies_to_category(df,multi_floor_cols,'multi_floor')

    df = df.drop(columns=['GEO_LATITUDINE_BENE_ROUNDED','GEO_LONGITUDINE_BENE_ROUNDED'])

    data_rev[key] = df
    del df




In [108]:
data_rev['df_real']

,flag_garage,flag_pertinenza,log_mq_bin,log_price_bin,CAP,energy_class,floor,cod_cat,costr_year,air_conditioning,multi_floor
0,1.0,0.0,1,2,60123,Missing_energy_class,floor_Missing,COD_CAT_A02,ANNO_COSTRUZIONE_1500_1965,flag_air_conditioning_Missing,flag_multi_floor_Missing
1,0.0,0.0,0,0,60025,Missing_energy_class,floor_Missing,COD_CAT_A02,ANNO_COSTRUZIONE_1965_1985,flag_air_conditioning_Missing,flag_multi_floor_Missing
2,0.0,0.0,4,1,60025,Missing_energy_class,floor_Missing,COD_CAT_A03,ANNO_COSTRUZIONE_1500_1965,flag_air_conditioning_Missing,flag_multi_floor_Missing
3,1.0,0.0,2,2,60012,Low_energy_class,floor_Missing,COD_CAT_A02,ANNO_COSTRUZIONE_1985_2005,flag_air_conditioning_Missing,flag_multi_floor_Missing
4,0.0,0.0,1,1,60124,Missing_energy_class,floor_Missing,COD_CAT_A02,ANNO_COSTRUZIONE_1500_1965,flag_air_conditioning_Missing,flag_multi_floor_Missing
...,...,...,...,...,...,...,...,...,...,...,...
4247,1.0,0.0,2,3,60020,Medium_energy_class,floor_Missing,COD_CAT_A02,ANNO_COSTRUZIONE_1985_2005,flag_air_conditioning_Missing,flag_multi_floor_Missing
4248,1.0,0.0,4,3,60020,Medium_energy_class,floor_Missing,COD_CAT_A02,ANNO_COSTRUZIONE_1985_2005,flag_air_conditioning_Missing,flag_multi_floor_Missing
4249,1.0,0.0,3,4,60020,High_energy_class,floor_Missing,COD_CAT_A02,ANNO_COSTRUZIONE_2005_2025,flag_air_conditioning_Missing,flag_multi_floor_Missing
4250,0.0,0.0,3,1,60020,Low_energy_class,floor_Missing,COD_CAT_A02,ANNO_COSTRUZIONE_1500_1965,flag_air_conditioning,flag_multi_floor_Missing


## Conto il numero di classi uniche per CAP
all'interno di questa sezione, conto (per ogni colonna) il numero di classi uniche per ogni CAP

In [109]:
count_cap = dict()
for key in data_rev.keys():
    df = data_rev[key].copy()
    res = pd.DataFrame(columns=['flag_garage','flag_pertinenza','log_mq_bin','log_price_bin',
                                'energy_class','floor','cod_cat','costr_year','air_conditioning','multi_floor'])
    for cap in df.CAP.unique():
        for c in res.columns:
            res.loc[cap,c] = len(df.loc[df.CAP == cap, c].unique())

    count_cap[key] = res


In [110]:
count_cap['df_real'].floor.value_counts()

floor
2    13
1     7
4     7
5     6
3     4
6     1
Name: count, dtype: int64

In [111]:
count_cap['df_nfvae'].floor.value_counts()

floor
1    38
Name: count, dtype: int64

In [112]:
count_cap.keys()

dict_keys(['df_real', 'df_nfvae', 'df_ablation', 'df_ipf', 'df_copula_nf', 'df_copula_ablation'])

In [113]:
for key in count_cap.keys():
    count_cap[key] = count_cap[key].sort_index()

In [114]:
count_cap['df_real']

,flag_garage,flag_pertinenza,log_mq_bin,log_price_bin,energy_class,floor,cod_cat,costr_year,air_conditioning,multi_floor
60010,2,2,5,5,4,2,4,4,2,2
60011,2,2,5,5,3,1,3,4,2,1
60012,2,2,5,5,4,2,4,4,2,2
60013,2,2,5,5,4,2,3,4,2,2
60015,2,2,5,5,4,4,4,4,2,2
60018,2,2,5,5,4,2,4,4,2,2
60019,2,2,5,5,4,5,4,4,2,2
60020,2,2,5,5,4,3,4,4,2,2
60021,2,2,5,5,4,1,4,5,2,1
60022,2,2,5,5,4,2,4,4,2,2


In [115]:
count_cap['df_nfvae']

,flag_garage,flag_pertinenza,log_mq_bin,log_price_bin,energy_class,floor,cod_cat,costr_year,air_conditioning,multi_floor
60010,2,2,5,5,4,1,3,4,2,2
60011,1,2,1,2,2,1,1,3,1,1
60012,2,2,5,5,4,1,2,4,2,2
60013,2,2,4,3,3,1,2,4,2,1
60015,2,2,5,5,4,1,4,4,2,2
60018,2,2,5,5,4,1,3,4,2,2
60019,2,2,5,5,4,1,4,4,2,2
60020,2,2,5,5,4,1,4,4,2,2
60021,2,2,5,5,3,1,4,4,2,2
60022,2,2,5,5,3,1,3,4,2,2


Non sembra andare molto bene, infatti il nostro modo (nf+vae) sembra preferirie troppo spesso le classi maggioritario (es floor)

## Conta case uniche per CAP

In [116]:
data_rev.keys()

dict_keys(['df_real', 'df_nfvae', 'df_ablation', 'df_ipf', 'df_copula_nf', 'df_copula_ablation'])

In [117]:
count_cap1 = pd.DataFrame(columns=['df_real', 'df_nfvae', 'df_ablation', 'df_ipf', 'df_copula_nf', 'df_copula_ablation'])

for cap in data_rev['df_real'].CAP.unique():
    for pop in count_cap1.columns:
        # number of houses in the CAP for the given syn pop
        df = data_rev[pop].copy()
        df_cap = df.loc[df.CAP == cap,:]
        N_homes_cap = 1 #len(df_cap)

        # number of unique homes in the CAP
        unique_homes = df_cap.drop_duplicates()
        N_unique_homes = len(unique_homes)

        if N_homes_cap != 0:
            count_cap1.loc[cap,pop] = N_unique_homes/N_homes_cap
        else:
            count_cap1.loc[cap,pop] = np.nan


In [118]:
count_cap1 = count_cap1.sort_index()
count_cap1

,df_real,df_nfvae,df_ablation,df_ipf,df_copula_nf,df_copula_ablation
60010,64.0,64.0,17.0,41.0,61.0,219.0
60011,15.0,3.0,4.0,7.0,27.0,159.0
60012,66.0,71.0,142.0,44.0,69.0,48.0
60013,28.0,19.0,17.0,19.0,33.0,89.0
60015,187.0,105.0,110.0,123.0,244.0,101.0
60018,88.0,59.0,77.0,51.0,89.0,70.0
60019,320.0,206.0,195.0,232.0,327.0,195.0
60020,164.0,106.0,135.0,105.0,191.0,305.0
60021,59.0,43.0,39.0,41.0,54.0,81.0
60022,149.0,83.0,40.0,92.0,167.0,107.0


In [93]:
data_rev.keys()

dict_keys(['df_real', 'df_nfvae', 'df_ablation', 'df_ipf', 'df_copula_nf', 'df_copula_ablation'])

In [94]:
data_rev['df_real']

,flag_garage,flag_pertinenza,log_mq_bin,log_price_bin,CAP,energy_class,floor,cod_cat,costr_year,air_conditioning,multi_floor
0,1.0,0.0,1,2,60123,Missing_energy_class,floor_Missing,COD_CAT_A02,ANNO_COSTRUZIONE_1500_1965,flag_air_conditioning_Missing,flag_multi_floor_Missing
1,0.0,0.0,0,0,60025,Missing_energy_class,floor_Missing,COD_CAT_A02,ANNO_COSTRUZIONE_1965_1985,flag_air_conditioning_Missing,flag_multi_floor_Missing
2,0.0,0.0,4,1,60025,Missing_energy_class,floor_Missing,COD_CAT_A03,ANNO_COSTRUZIONE_1500_1965,flag_air_conditioning_Missing,flag_multi_floor_Missing
3,1.0,0.0,2,2,60012,Low_energy_class,floor_Missing,COD_CAT_A02,ANNO_COSTRUZIONE_1985_2005,flag_air_conditioning_Missing,flag_multi_floor_Missing
4,0.0,0.0,1,1,60124,Missing_energy_class,floor_Missing,COD_CAT_A02,ANNO_COSTRUZIONE_1500_1965,flag_air_conditioning_Missing,flag_multi_floor_Missing
...,...,...,...,...,...,...,...,...,...,...,...
4247,1.0,0.0,2,3,60020,Medium_energy_class,floor_Missing,COD_CAT_A02,ANNO_COSTRUZIONE_1985_2005,flag_air_conditioning_Missing,flag_multi_floor_Missing
4248,1.0,0.0,4,3,60020,Medium_energy_class,floor_Missing,COD_CAT_A02,ANNO_COSTRUZIONE_1985_2005,flag_air_conditioning_Missing,flag_multi_floor_Missing
4249,1.0,0.0,3,4,60020,High_energy_class,floor_Missing,COD_CAT_A02,ANNO_COSTRUZIONE_2005_2025,flag_air_conditioning_Missing,flag_multi_floor_Missing
4250,0.0,0.0,3,1,60020,Low_energy_class,floor_Missing,COD_CAT_A02,ANNO_COSTRUZIONE_1500_1965,flag_air_conditioning,flag_multi_floor_Missing


In [120]:
data_rev['df_ipf'].loc[0,:].isin(data_rev['df_real'])

flag_garage         False
flag_pertinenza     False
log_mq_bin          False
log_price_bin       False
CAP                 False
energy_class        False
floor               False
cod_cat             False
costr_year          False
air_conditioning    False
multi_floor         False
Name: 0, dtype: bool

Osservazioni nei dataframe binnati che esistono nei dati sintetici ma non ci sono nei dati reali

In [131]:
from sklearn.metrics import pairwise_distances
import gower

def TableIMS(data, control_data, metric='euclidean'):
    res = pd.DataFrame(columns=['equal_homes','different_homes'])
    for i in control_data:
        # Distance matrix
        if(metric=='euclidean'):
            train_dists = pairwise_distances(data[i].values, data['df_real'].values, metric='euclidean') 
        elif(metric=='norm1'):
            train_dists = pairwise_distances(data[i].values, data['df_real'].values, metric='minkowski', p=1) 
        elif(metric == 'gower'):
            train_dists = gower.gower_matrix(data[i],data['df_real'] )
        
        # take the minimum distances
        min_dists_train = train_dists.min(axis=1)

        res.loc[i,:] = [np.sum(min_dists_train==0), np.sum(min_dists_train!=0)]

    return res

In [132]:
data_rev.keys()

dict_keys(['df_real', 'df_nfvae', 'df_ablation', 'df_ipf', 'df_copula_nf', 'df_copula_ablation'])

In [133]:
control = ['df_nfvae', 'df_ablation', 'df_ipf', 'df_copula_nf', 'df_copula_ablation']
res = TableIMS(data_rev,control,metric='gower')

In [134]:
res

,equal_homes,different_homes
df_nfvae,1002,3250
df_ablation,823,3429
df_ipf,1820,2432
df_copula_nf,516,3736
df_copula_ablation,411,3841


In [137]:
res['sum'] = res['equal_homes'] + res['different_homes']
res

,equal_homes,different_homes,sum
df_nfvae,1002,3250,4252
df_ablation,823,3429,4252
df_ipf,1820,2432,4252
df_copula_nf,516,3736,4252
df_copula_ablation,411,3841,4252


In [135]:
len(data_rev['df_real'])

4252

In [136]:
1820/4252

0.4280338664158043